**Report of Project 3: "Implementation and evaluation of Otsu Thresholding"**

*presented by Elizaveta Chernova, Veronika Schuler, Laura Wächter, Hannah Winter*

**Abstract**

Otsu Thresholding is a valuable method in detecting the ideal threshold of an image. It is therefore used frequently in
image segmentation for biological and medical purposes. Here we use the implementation of this algorithm for
segmentation of cell nuclei from the datasets N2DH-GOWT1 and N2DL-HeLa cells of the cell tracking challenge and the
NIH3T3 images of Coelho and colleagues in their challenge of nuclear segmentation in microscope cell images. For
pre-processing the images, we used histogram stretching for images with low resolution, a gaussian filter, and a median
filter as well as twolevel Otsu Thresholding for excluding reflections in some images. The implemented Otsu algorithm
used on the pre-processed images was then evaluated with the Dice score, the median surface distance function and the
Hausdorff metric. We find that...

**Table of contents**

...

**Introduction**

In computer vision, image segmentation is applied to a variety of challenges. Those include detecting cell sizes
and the number of cells in high-throughput cytometry or for cell tracking as well as segmentation of brain images.

Challenges that need to be tackled in finding the correct threshold for image segmentation are low contrast images,
reflections and random noise that can occur in the images. A starting point would be the preprocessing of the images
before using the Otsu algorithm.

The performance of the algorithm can be evaluated using the dice score as well as the median surface distance function
and the Hausdorff metric, that compare the segmented image to the hand-segmented ground truth image.

Furthermore, to give a purpose to the segmented images, we implemented a cell counting algorithm to determine the number
of cells in an image. In addition to the performance measurement of the algorithm, we visualize the results with an
overlay of the segmented image and the ground truth image.

**Description of the datasets**

N2DH-GOWT1 cells

The dataset N2DH-GOWT1 ot the cell tracking challenge (Bártová et al., 2011) contain images of GFP-Gowt1 mouse embryonic
stem cells that have been derived with time-lapse confocal microscopy with a Leica TCS SP5 microscope. The varying
brightness of the cells makes it hard to distinguish all the cells from the background.

N2HL-HeLa cells

The dataset N2DL-HeLa of the cell tracking challenge (Neumann et al., 2010) contains images of human epithelial cells
of cervical cancer. Those images have been derived with an Olympus IX81 microscope used for live imaging of fluorescently
labelled chromosomes. The challenge in those images is the varying brightness of cells.

NIH3T3 cells 

The dataset NIH3T3 (Coelho et al., 2009) contains images of several mouse embryonic fibroblast cells. Those images have also 
been derived with fluorescence microscopy images and the difficulty in segmenting those images mainly lies in the bright
light spots, probably from the microscope used, that make it difficult for the algorithm to choose a threshold between
the brightness of the cells and the background and not between the brightness of light spots and the cells.

**Import of modules**

In [2]:
from skimage.io import imread
from skimage.io import imshow
from matplotlib import pyplot as plt
from skimage.filters import threshold_otsu
from nuclei_segmentation import pathlist
from nuclei_segmentation import otsu
from nuclei_segmentation import evaluation
import pathlib as pl

**Loading images, Preprocessing, Otsu, Evaluation of usage on datasets, Discussion**

In [5]:
image = imread(str(pl.Path(r'Data\NIH3T3\img\dna-27.png')))
thol, gness = otsu.otsu(image)
print(thol, gness)

43 0.4985326808412964


In [8]:
#otsu.otsu(image)


In [ ]:
#threshold_otsu(image)

**This is our pipeline assembly used on all pictures**


In [11]:
#1st dataset N2DH_GOWT1
img_N2DH_GOWT1_dir = ['Data/N2DH-GOWT1/img/*.tif']
img_N2DH_GOWT1 = imread_collection(img_N2DH_GOWT1_dir, plugin='tifffile')

gt_N2DH_GOWT1_dir = ['Data/N2DH-GOWT1/gt/*.tif']
gt_N2DH_GOWT1 = imread_collection(gt_N2DH_GOWT1_dir, plugin='tifffile')

#an example of accessing a single image from a list 
print(img_N2DH_GOWT1[0])
plt.imshow(img_N2DH_GOWT1[0], "gray")
plt.show()

#path:
clipped_list_N2DH_GOWT1 = []
gt_list_N2DH_GOWT1 = []

for i in img_N2DH_GOWT1:
    threshold, goodness = otsu.otsu_faster(i)
    clipped_list_N2DH_GOWT1.append(otsu.clipping(i,threshold))

for i in gt_N2DH_GOWT1:
    gt_list_N2DH_GOWT1.append(i)

for i in range(len(clipped_list_N2DH_GOWT1)):
    print(evaluation.dice_score_faster(clipped_list_N2DH_GOWT1[i], gt_list_N2DH_GOWT1[i]))
    
    


#3rd dataset NIH3T3
img_NIH3T3_dir = ['Data/NIH3T3/img/*.png']
img_NIH3T3 = imread_collection(img_NIH3T3_dir)

gt_NIH3T3_dir = ['Data/NIH3T3/gt/*.png']
gt_NIH3T3 = imread_collection(gt_NIH3T3_dir)

#path:
clipped_list_NIH3T3 = []
gt_list_NIH3T3 = []

for i in img_NIH3T3:
    threshold, goodness = otsu.otsu_faster(i)
    clipped_list_NIH3T3.append(otsu.clipping(i,threshold))

for i in gt_NIH3T3:
    gt_list_NIH3T3.append(i)

for i in range(len(clipped_list_NIH3T3)):
    print(evaluation.dice_score_faster(clipped_list_NIH3T3[i], gt_list_NIH3T3[i]))



SyntaxError: cannot assign to operator (<ipython-input-11-2d3188d25d4b>, line 4)

In [12]:
#using otsu thresholding on a single image
otsu_threshold = otsu_faster(img)
otsu_threshold  

NameError: name 'otsu_faster' is not defined

In [13]:
# dice score
dice_score_faster(clipped, gt_N2DH-GOWT1)

NameError: name 'dice_score_faster' is not defined

In [51]:
# IoU
IoU(clipped,gt_N2DH-GOWT1)


0.07139965198215116